<a href="https://colab.research.google.com/github/omidasudeh/Interview_practice/blob/master/SafeAutoChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

## Load the data

In [0]:
raw_data= pd.read_csv('./safeAuto_data_challenge.csv')

/home/omid/miniconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Sense making and Cleaning

---

## First Look

In [0]:
print 'raw data Shape:'+ str(raw_data.shape)
print "==========\n"
print "raw data data-types:\n"
print  raw_data.dtypes
raw_data.head()

raw data Shape:(214770, 13)

raw data data-types:

InteractionId       object
UserId              object
State               object
UserAgentString     object
Browser             object
BrowserVersion      object
DeviceIsMobile      object
DeviceBrand         object
ScreenWidth        float64
ScreenHeight       float64
EventType           object
Event               object
RecordDateTime      object
dtype: object


,InteractionId,UserId,State,UserAgentString,Browser,BrowserVersion,DeviceIsMobile,DeviceBrand,ScreenWidth,ScreenHeight,EventType,Event,RecordDateTime
0,53981793,0x00028751F53A023C6162199777A9A6F0,SC,Mozilla/5.0 (Linux; Android 5.0.2; SM-S920L Bu...,Chrome,38.0,True,Linux,360.0,511.0,Web Routing,Pre-Quote Portal,2016-10-09 12:16:53.7530000
1,53981793,0x00028751F53A023C6162199777A9A6F0,SC,Mozilla/5.0 (Linux; Android 5.0.2; SM-S920L Bu...,Chrome,38.0,True,Linux,360.0,511.0,Web Routing,Retrieve Existing Policy,2016-10-09 12:17:31.5300000
2,53981793,0x00028751F53A023C6162199777A9A6F0,SC,Mozilla/5.0 (Linux; Android 5.0.2; SM-S920L Bu...,Chrome,38.0,True,Linux,360.0,511.0,Web Routing,Quote Start,2016-10-09 12:19:31.1270000
3,53981793,0x00028751F53A023C6162199777A9A6F0,SC,Mozilla/5.0 (Linux; Android 5.0.2; SM-S920L Bu...,Chrome,38.0,True,Linux,360.0,511.0,Web Routing,Retrieve Premium,2016-10-09 12:22:36.2930000
4,53981793,0x00028751F53A023C6162199777A9A6F0,SC,Mozilla/5.0 (Linux; Android 5.0.2; SM-S920L Bu...,Chrome,38.0,True,Linux,360.0,511.0,Web Routing,Bind Start,2016-10-09 12:23:40.8300000


## Run some basic statistical summaries on the data


In [0]:
raw_data.describe(include = 'all')

,InteractionId,UserId,State,UserAgentString,Browser,BrowserVersion,DeviceIsMobile,DeviceBrand,ScreenWidth,ScreenHeight,EventType,Event,RecordDateTime
count,214770,214770,214401,214401,214401,214401.0,214401,214401,214401.000000,214401.000000,214401,214401,214032
unique,43103,11839,16,4342,8,97.0,2,11,NaN,NaN,3,28,213276
top,54028356,0x72C4544F0E010DD2512C3BA1076B7559,OH,Mozilla/5.0 (iPhone; CPU iPhone OS 10_0_2 like...,Chrome,53.0,True,Linux,NaN,NaN,Web Routing,Pre-Quote Portal,2016-10-07 14:40:49.1030000
freq,789,831,29397,11957,145322,58838.0,126829,90041,NaN,NaN,208019,47939,3
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,796.422689,633.773807,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,551.435277,155.179256,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,360.000000,520.000000,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,412.000000,604.000000,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1366.000000,685.000000,NaN,NaN,NaN


## Attributes notes:
### The list of categorical attributes: (few categories, no order)
Event:28, EventType:3, DeviceBrand:11, Browser:8, BrowserVersion:97, State:16 
### need to convert time to time series
- InteractionId,UserId repeated 214770 times
- time repeated 214032 times
- other repeated 214401
- (Browser	BrowserVersion)(DeviceIsMobile	DeviceBrand	ScreenWidth	ScreenHeight)(EventType	Event)(RecordDateTime)

In [0]:
categ_attrs = ['State', 'Browser', 'BrowserVersion','DeviceIsMobile','DeviceBrand', 'EventType', 'Event']

### Check for NULL

In [0]:
print raw_data.isnull().sum()

InteractionId        0
UserId               0
State              369
UserAgentString    369
Browser            369
BrowserVersion     369
DeviceIsMobile     369
DeviceBrand        369
ScreenWidth        369
ScreenHeight       369
EventType          369
Event              369
RecordDateTime     738
dtype: int64


### Get the possible values for each  catgorical attribute

In [0]:
for attr in categ_attrs:
    print attr,":"
    line = ""
    for item in raw_data[attr].unique():
        line = line+ str(item)+"\t"
    print line
    print "========================="

State :
SC	OK	AZ	VA	OH	PA	TX	GA	LA	IN	nan	KY	MO	MS	IL	TN	KS	
Browser :
Chrome	Mozilla	Safari	InternetExplorer	nan	Firefox	IEMobile	IE	Unknown	
BrowserVersion :
38.0	0.0	53.0	10.0	46.0	51.0	36.0	11.0	44.0	9.0	43.0	52.0	49.0	nan	9.1	47.0	5.1	50.0	45.0	35.0	48.0	30.0	33.0	8.0	39.0	34.0	28.0	42.0	7.0	37.0	41.0	54.0	6.2	40.0	10.3	31.0	6.0	18.0	21.0	55.0	20.2	44.0a1	10.2	38.9	32.2	26.0	39.4	44.0	53.0	9.0	11.0	46.0	9.1	49.0	38.0	43.0	10.0	45.0	0.0	50.0	52.0	30.0	51.0	42.0	31.0	40.0	39.0	47.0	8.0	35.0	37.0	36.0	48.0	33.0	18.0	28.0	54.0	34.0	55.0	6.2	7.0	5.1	10.3	32.0	5.0	6.1	41.0	24.0	32.2	17.0	22.0	16.0	26.0	40.1	44.4	6.0	29.0	21.0	
DeviceIsMobile :
True	False	nan	
DeviceBrand :
Linux	iPhone	IPhone	Unknown	IPad	nan	Mobile	iPad	Windows Phone 10.0	BB10	IPod	iPod touch	
EventType :
Web Routing	Client Error	Server Error	nan	
Event :
Pre-Quote Portal	Retrieve Existing Policy	Quote Start	Retrieve Premium	Bind Start	Add Drivers	Add Vehicles	Payment Start	Payment Complete	TokenGuid is not valid	nan	D

- EventType can be: 'TokenGuid is not valid', 'Referred to Phone Rep' '#/404' and more
- Event can be: 'Web Routing' 'Client Error' 'Server Error' nan
- DeviceBrand can be Unknown

## DataFrame Cleaning raw_data ==> data1
- convert RecordDateTime to pandas time
- remove nan entries

In [0]:
data1 = raw_data.copy()
data1['RecordDateTime'] = pd.to_datetime(data1['RecordDateTime'])
data1.dropna(inplace= True)
data1.dtypes

InteractionId              object
UserId                     object
State                      object
UserAgentString            object
Browser                    object
BrowserVersion             object
DeviceIsMobile             object
DeviceBrand                object
ScreenWidth               float64
ScreenHeight              float64
EventType                  object
Event                      object
RecordDateTime     datetime64[ns]
dtype: object

### Keep only 'Web Routing' EventType and do not consider Error EventTypes

In [0]:
data1_noError = data1[data1['EventType']=="Web Routing"].copy()

### Select only the folowing columns for analysis
- 'InteractionId',
- 'UserId'
- 'Event'
- 'RecordDateTime'] 

In [0]:
data1_subset = data1_noError[['InteractionId','UserId','Event','RecordDateTime']].copy()
print "number of rows: " + str(data1_subset.shape[0])
print "number of payments complete: " + str(data1_subset[data1_subset['Event']== 'Payment Complete'].shape[0])

number of rows: 208019
number of payments complete: 5311


In [0]:
nans = lambda df: df[df.isnull().any(axis=1)]
nans(data1_subset)

,InteractionId,UserId,Event,RecordDateTime


## Compute Event Duration

In [0]:
tbl = data1_subset.copy()
tbl['Event_duration'] = tbl.groupby(['UserId','InteractionId']).RecordDateTime.apply(lambda x: x.shift(-1) -x )
tbl['Event_duration'] = tbl.Event_duration.astype('timedelta64[s]')/60
del tbl['RecordDateTime']

In [0]:
tbl1 = tbl.copy()

In [0]:
nans(tbl1).shape[0]

43091

In [0]:
tbl1.head()

,InteractionId,UserId,Event,Event_duration
0,53981793,0x00028751F53A023C6162199777A9A6F0,Pre-Quote Portal,0.616667
1,53981793,0x00028751F53A023C6162199777A9A6F0,Retrieve Existing Policy,1.983333
2,53981793,0x00028751F53A023C6162199777A9A6F0,Quote Start,3.083333
3,53981793,0x00028751F53A023C6162199777A9A6F0,Retrieve Premium,1.066667
4,53981793,0x00028751F53A023C6162199777A9A6F0,Bind Start,0.783333


### duration computation makes lots of nans ==> remove nans

In [0]:
tbl1.dropna(inplace= True)
nans(tbl1).shape[0]

0

In [0]:

print "number of rows before dropna: " + str(tbl.shape[0])
print "number of rows after dropna: " + str(tbl1.shape[0])

print "number of payments complete before dropna: " + str(tbl[tbl['Event']== 'Payment Complete'].shape[0])
print "number of payments complete after dropna: " + str(tbl1[tbl1['Event']== 'Payment Complete'].shape[0])

number of rows before dropna: 208019
number of rows after dropna: 164928
number of payments complete before dropna: 5311
number of payments complete after dropna: 2492


# Data size after duration computation
- number of rows: 164928
- number of payments complete: 2492

In [0]:
print "number of rows: " + str(tbl1.shape[0])
print "number of payments complete: " + str(tbl1[tbl1['Event']== 'Payment Complete'].shape[0])

number of rows: 164928
number of payments complete: 2492


In [0]:
tbl1.head()

,InteractionId,UserId,Event,Event_duration
0,53981793,0x00028751F53A023C6162199777A9A6F0,Pre-Quote Portal,0.616667
1,53981793,0x00028751F53A023C6162199777A9A6F0,Retrieve Existing Policy,1.983333
2,53981793,0x00028751F53A023C6162199777A9A6F0,Quote Start,3.083333
3,53981793,0x00028751F53A023C6162199777A9A6F0,Retrieve Premium,1.066667
4,53981793,0x00028751F53A023C6162199777A9A6F0,Bind Start,0.783333


# Pivoing


---



## Pivot groups of ['UserId','InteractionId','Event'] on Event 

In [0]:
tbl_copy = tbl1.copy()
pivot_tbl = tbl_copy.groupby(['UserId','InteractionId','Event'])['Event_duration'].sum().to_frame('Event_duration').reset_index().pivot_table(index = ['UserId','InteractionId'],columns = 'Event', values = 'Event_duration')
pivot_tbl.head()


Event                                             #/404  #/howtopay  \
UserId                             InteractionId                      
0x00028751F53A023C6162199777A9A6F0 53981793         NaN         NaN   
                                   54054753         NaN         NaN   
0x000C03C8BF9ACCB2785B239703D91B48 53900548         NaN         NaN   
                                   54049776         NaN         NaN   
                                   54103835         NaN         NaN   

Event                                             Add Drivers  Add Vehicles  \
UserId                             InteractionId                              
0x00028751F53A023C6162199777A9A6F0 53981793               NaN           NaN   
                                   54054753               NaN           NaN   
0x000C03C8BF9ACCB2785B239703D91B48 53900548               NaN           NaN   
                                   54049776          0.033333          1.25   
                                   54103835               NaN           NaN   

Event                                             Bind Start  Choose Coverage  \
UserId                             InteractionId                                
0x00028751F53A023C6162199777A9A6F0 53981793         0.783333              NaN   
                                   54054753              NaN              NaN   
0x000C03C8BF9ACCB2785B239703D91B48 53900548              NaN              NaN   
                                   54049776         0.050000              NaN   
                                   54103835              NaN              NaN   

Event                                             Download Receipt  \
UserId                             InteractionId                     
0x00028751F53A023C6162199777A9A6F0 53981793                    NaN   
                                   54054753                    NaN   
0x000C03C8BF9ACCB2785B239703D91B48 53900548                    NaN   
                                   54049776                    NaN   
                                   54103835                    NaN   

Event                                             Get Premium  \
UserId                             InteractionId                
0x00028751F53A023C6162199777A9A6F0 53981793               NaN   
                                   54054753               NaN   
0x000C03C8BF9ACCB2785B239703D91B48 53900548               NaN   
                                   54049776               NaN   
                                   54103835               NaN   

Event                                             Payment Complete  \
UserId                             InteractionId                     
0x00028751F53A023C6162199777A9A6F0 53981793                    NaN   
                                   54054753                    NaN   
0x000C03C8BF9ACCB2785B239703D91B48 53900548                    NaN   
                                   54049776                    NaN   
                                   54103835                    NaN   

Event                                             Payment Start  \
UserId                             InteractionId                  
0x00028751F53A023C6162199777A9A6F0 53981793                 NaN   
                                   54054753                 NaN   
0x000C03C8BF9ACCB2785B239703D91B48 53900548                 NaN   
                                   54049776            0.566667   
                                   54103835                 NaN   

Event                                             Pre-Quote Portal  \
UserId                             InteractionId                     
0x00028751F53A023C6162199777A9A6F0 53981793               0.616667   
                                   54054753               0.350000   
0x000C03C8BF9ACCB2785B239703D91B48 53900548               2.366667   
                                   54049776               0.516667   
                                   54103835         

## Data Cleaning on pivot_tbl

In [0]:
del pivot_tbl['#/404']
del pivot_tbl['#/howtopay']
del pivot_tbl['Referred to Phone Rep']
pivot_tbl = pivot_tbl.fillna(0)
pivot_tbl.head()

Event                                             Add Drivers  Add Vehicles  \
UserId                             InteractionId                              
0x00028751F53A023C6162199777A9A6F0 53981793          0.000000          0.00   
                                   54054753          0.000000          0.00   
0x000C03C8BF9ACCB2785B239703D91B48 53900548          0.000000          0.00   
                                   54049776          0.033333          1.25   
                                   54103835          0.000000          0.00   

Event                                             Bind Start  Choose Coverage  \
UserId                             InteractionId                                
0x00028751F53A023C6162199777A9A6F0 53981793         0.783333              0.0   
                                   54054753         0.000000              0.0   
0x000C03C8BF9ACCB2785B239703D91B48 53900548         0.000000              0.0   
                                   54049776         0.050000              0.0   
                                   54103835         0.000000              0.0   

Event                                             Download Receipt  \
UserId                             InteractionId                     
0x00028751F53A023C6162199777A9A6F0 53981793                    0.0   
                                   54054753                    0.0   
0x000C03C8BF9ACCB2785B239703D91B48 53900548                    0.0   
                                   54049776                    0.0   
                                   54103835                    0.0   

Event                                             Get Premium  \
UserId                             InteractionId                
0x00028751F53A023C6162199777A9A6F0 53981793               0.0   
                                   54054753               0.0   
0x000C03C8BF9ACCB2785B239703D91B48 53900548               0.0   
                                   54049776               0.0   
                                   54103835               0.0   

Event                                             Payment Complete  \
UserId                             InteractionId                     
0x00028751F53A023C6162199777A9A6F0 53981793                    0.0   
                                   54054753                    0.0   
0x000C03C8BF9ACCB2785B239703D91B48 53900548                    0.0   
                                   54049776                    0.0   
                                   54103835                    0.0   

Event                                             Payment Start  \
UserId                             InteractionId                  
0x00028751F53A023C6162199777A9A6F0 53981793            0.000000   
                                   54054753            0.000000   
0x000C03C8BF9ACCB2785B239703D91B48 53900548            0.000000   
                                   54049776            0.566667   
                                   54103835            0.000000   

Event                                             Pre-Quote Portal  \
UserId                             InteractionId                     
0x00028751F53A023C6162199777A9A6F0 53981793               0.616667   
                                   54054753               0.350000   
0x000C03C8BF9ACCB2785B239703D91B48 53900548               2.366667   
                                   54049776               0.516667   
                                   54103835               0.600000   

Event                                             Quote Start  \
UserId                             InteractionId                
0x00028751F53A023C6162199777A9A6F0 53981793          3.083333   
                                   54054753          0.000000   
0x000C03C8BF9ACCB2785B239703D91B48 53900548          0.000000   
                                   54049776          0.000000   
                                   54103835          0.000000   

Event                  

In [0]:
print "number of rows: "+ str(pivot_tbl.shape[0])
print "number of complete payments: "+str(pivot_tbl[pivot_tbl['Payment Complete']>0].shape[0])

number of rows: 38786
number of complete payments: 2022


## Coverting 'Payment Complete' values to {0,1} as the outcome class
 - Also Remove the 'Payment Complete' column and keep it as our labels vector
 
 - Remove the 'Download Receipt' attribute as it is after payment complete

In [0]:
pivot_tbl_1 = pivot_tbl.copy()
pivot_tbl_1[pivot_tbl_1['Payment Complete']>0] = 1
Payment_complete = pivot_tbl_1['Payment Complete'].copy()
del pivot_tbl_1['Payment Complete']
del pivot_tbl_1['Download Receipt']
pivot_tbl_1.head(10)

Event                                             Add Drivers  Add Vehicles  \
UserId                             InteractionId                              
0x00028751F53A023C6162199777A9A6F0 53981793          0.000000          0.00   
                                   54054753          0.000000          0.00   
0x000C03C8BF9ACCB2785B239703D91B48 53900548          0.000000          0.00   
                                   54049776          0.033333          1.25   
                                   54103835          0.000000          0.00   
0x001647CB1A1BD8DFCA6A94C4DBA06DEB 53716828          5.850000          0.00   
0x004D6AD7358B2140229AD0E5605159F9 53709709          0.000000          0.00   
                                   53839756          0.000000          0.00   
0x0053B38B4B5CB1FC8A6B13F891473B24 53801959          0.000000          0.00   
                                   53815875        131.816667          1.60   

Event                                             Bind Start  Choose Coverage  \
UserId                             InteractionId                                
0x00028751F53A023C6162199777A9A6F0 53981793         0.783333              0.0   
                                   54054753         0.000000              0.0   
0x000C03C8BF9ACCB2785B239703D91B48 53900548         0.000000              0.0   
                                   54049776         0.050000              0.0   
                                   54103835         0.000000              0.0   
0x001647CB1A1BD8DFCA6A94C4DBA06DEB 53716828         7.950000              0.0   
0x004D6AD7358B2140229AD0E5605159F9 53709709         0.000000              0.0   
                                   53839756         0.000000              0.0   
0x0053B38B4B5CB1FC8A6B13F891473B24 53801959         0.000000              0.0   
                                   53815875         0.066667              0.0   

Event                                             Get Premium  Payment Start  \
UserId                             InteractionId                               
0x00028751F53A023C6162199777A9A6F0 53981793               0.0       0.000000   
                                   54054753               0.0       0.000000   
0x000C03C8BF9ACCB2785B239703D91B48 53900548               0.0       0.000000   
                                   54049776               0.0       0.566667   
                                   54103835               0.0       0.000000   
0x001647CB1A1BD8DFCA6A94C4DBA06DEB 53716828               0.0       0.000000   
0x004D6AD7358B2140229AD0E5605159F9 53709709               0.0       0.000000   
                                   53839756               0.0       0.000000   
0x0053B38B4B5CB1FC8A6B13F891473B24 53801959               0.0       0.000000   
                                   53815875               0.0       0.000000   

Event                                             Pre-Quote Portal  \
UserId                             InteractionId                     
0x00028751F53A023C6162199777A9A6F0 53981793               0.616667   
                                   54054753               0.350000   
0x000C03C8BF9ACCB2785B239703D91B48 53900548               2.366667   
                                   54049776               0.516667   
                                   54103835               0.600000   
0x001647CB1A1BD8DFCA6A94C4DBA06DEB 53716828               1.683333   
0x004D6AD7358B2140229AD0E5605159F9 53709709               3.083333   
                                   53839756               1.933333   
0x0053B38B4B5CB1FC8A6B13F891473B24 53801959               5.600000   
                                   53815875               0.550000   

Event                                             Quote Start  \
UserId                             InteractionId                
0x00028751F53A023C6162199777A9A6F0 53981793          3.083333   
                                   54054753          0.000000   
0x000C03C8BF9ACC

# Model Traning
 - build a list of events
 - remove events columns one by one, normalize the duration of the remainings, and train different models
 - compute goodness measures (accuracy, precision, recall) for all the model

In [0]:
Events = ['Payment Start','Get Premium','Choose Coverage','Add Vehicles',
'Add Drivers','Bind Start','Retrieve Premium','Retrieve Existing Policy',
'Quote Start','Pre-Quote Portal']
print Events

['Payment Start', 'Get Premium', 'Choose Coverage', 'Add Vehicles', 'Add Drivers', 'Bind Start', 'Retrieve Premium', 'Retrieve Existing Policy', 'Quote Start', 'Pre-Quote Portal']


In [0]:
pivot_tbl_2 = pivot_tbl_1.copy()
for i in range(len(Events)):
    print "\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
    print "  Model: ",i+1
    print "%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%"
    
    print "training with only the following attributes:\n"
    print Events[i:len(Events)]
    #//////////////////////////////
    
    # Normalizing event durations
    data2 = pivot_tbl_2.copy()
    data2['sum'] = data2.sum(axis=1)
    for column in data2.columns:
        data2[column] = data2[column].div(data2['sum'])
    del data2['sum']
    data3 = data2.fillna(0).copy()
    #//////////////////////////////
    
    # Traning a model for complete payment using other all attributes
    X = data3.copy()
    y = Payment_complete
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state=12345)
    lr = LogisticRegression()
    lr.fit(X_train, y_train)
    print "\nAccuracy:" + str(lr.score(X_test,y_test))
    print "========================"
    y_pred = lr.predict(X_test)
    print(classification_report(y_test, y_pred))
    col_coef = zip(list(X),lr.coef_[0])
    print "Model Parameters:"
    print "========================"
    for col,coef in col_coef:
        print col+ " : " + str(coef)
    # ////////////////////////////
    
    del pivot_tbl_2[Events[i]]

    
    



%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  Model:  1
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
training with only the following attributes:

['Payment Start', 'Get Premium', 'Choose Coverage', 'Add Vehicles', 'Add Drivers', 'Bind Start', 'Retrieve Premium', 'Retrieve Existing Policy', 'Quote Start', 'Pre-Quote Portal']

Accuracy:0.996132691646614


/home/omid/miniconda2/lib/python2.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     11058
         1.0       0.93      1.00      0.96       578

   micro avg       1.00      1.00      1.00     11636
   macro avg       0.96      1.00      0.98     11636
weighted avg       1.00      1.00      1.00     11636

Model Parameters:
Add Drivers : -3.843448571286223
Add Vehicles : -3.903990968795953
Bind Start : -2.82077766727865
Choose Coverage : 30.783579886020302
Get Premium : 8.080519341651485
Payment Start : -2.592187526342751
Pre-Quote Portal : -7.249748200841941
Quote Start : -4.413346560901448
Retrieve Existing Policy : -2.475924376994149
Retrieve Premium : -4.500058019333033


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  Model:  2
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
training with only the following attributes:

['Get Premium', 'Choose Coverage', 'Add Vehicles', 'Add Drivers', 'Bind Start', 'Retrieve Premium', 'Retrieve Existing Policy', 'Quote Start', 'Pre-Quote Portal']

Accuracy:0.9

/home/omid/miniconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)




%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  Model:  5
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
training with only the following attributes:

['Add Drivers', 'Bind Start', 'Retrieve Premium', 'Retrieve Existing Policy', 'Quote Start', 'Pre-Quote Portal']

Accuracy:0.950326572705397
              precision    recall  f1-score   support

         0.0       0.95      1.00      0.97     11058
         1.0       0.00      0.00      0.00       578

   micro avg       0.95      0.95      0.95     11636
   macro avg       0.48      0.50      0.49     11636
weighted avg       0.90      0.95      0.93     11636

Model Parameters:
Add Drivers : 0.26149485235601017
Bind Start : 1.4795207127361452
Pre-Quote Portal : -3.3465150874453924
Quote Start : -0.364440321020779
Retrieve Existing Policy : 1.6543033403824488
Retrieve Premium : -0.14257221291202926


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
  Model:  6
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
training with only the following attributes:

['Bind Start', 'Retrieve Premium', 'Retrieve E